# Solutions to Modelling - Feature Engineering

# Exercise 1

We can train our model using the different datasets and feature engineering techniques to evaluate their impact on the model performance.
Print the score for the different methods we just introduced and compare them.

Copy this code to the original lecture notebook in order to execute it - it will not work here!

In [ ]:
""" print("Raw (but imputed) Features: {}".
      format(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)))
print("Normalized Features: {}".
      format(score_dataset(train_X_normalized, val_X_normalized, y_train, y_valid)))
print("Standardized Features: {}".
      format(score_dataset(train_X_standardized, val_X_standardized, y_train, y_valid)))
print("Log Landsize: {}".format(score_dataset(train_X_logGains, val_X_logGains, y_train, y_valid))) """

The lower the MAE is, the better. So in this case, transforming the Landsize to a logarithmic scale (only one column!) has the biggest effect on the performance. It should be noted that in general it is recommended to try different combinations of the possible transformations, not only one each.

# Exercise 2
Use standardized numeric features like in the previous section and combine it with OneHot Encoding and Label Encoding for training the model.

The final results should have two cases: 

- One-hot encoded categorical + standardized numeric
- Label encoded categorical + standardized numeric

For each of the considered categorical columns, does it make more sense to use label encoding or one-hot encoding from a data perspective?

Copy this code to the original lecture notebook in order to execute it - it will not work here!

In [ ]:
""" num_X_train = X_train.drop(low_cardinality_cols, axis=1)
num_X_valid = X_valid.drop(low_cardinality_cols, axis=1)

# standardize numerical columns
scaler = StandardScaler()
num_train_X_standardized = pd.DataFrame(scaler.fit_transform(num_X_train), 
                                        columns=num_X_train.columns, index=num_X_train.index)
num_val_X_standardized = pd.DataFrame(scaler.transform(num_X_valid), 
                                      columns=num_X_valid.columns, index=num_X_valid.index)


# One-hot encoding
one_hot_X_train = pd.concat([num_train_X_standardized, one_hot_cols_train], axis=1)
one_hot_X_valid = pd.concat([num_val_X_standardized, one_hot_cols_valid], axis=1)

# Evaluate performance
one_hot_encoding = score_dataset(one_hot_X_train.to_numpy(), one_hot_X_valid.to_numpy(), y_train, y_valid)
print("MAE using One-hot Encoding and Standardization: {}".format(one_hot_encoding))


# Label encoding
label_X_train_cat = pd.DataFrame(columns=low_cardinality_cols)
label_X_valid_cat = pd.DataFrame(columns=low_cardinality_cols)

label_encoder = LabelEncoder()
for col in low_cardinality_cols:
    label_X_train_cat[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid_cat[col] = label_encoder.transform(X_valid[col])

# align indices
label_X_train_cat.index = num_train_X_standardized.index
label_X_valid_cat.index = num_val_X_standardized.index

label_X_train = pd.concat([num_train_X_standardized, label_X_train_cat], axis=1)
label_X_valid = pd.concat([num_val_X_standardized, label_X_valid_cat], axis=1)

# Evaluate performance
label_encoding = score_dataset(label_X_train, label_X_valid, y_train, y_valid)
print("MAE using Label Encoding and Standardization: {}".format(mae_label_encoding)) """

All three columns should be encoded with a One-hot Encoder since their categories don't have an explicit ordering. This is also reflected in the MAE of both the model with the standardized and the non-standardized numerical columns.

## Exercise 3
1. If we have better information on the content of the categorical variables and time permits, we would not simply use the same encoder for all columns. Look at all columns of type "object" of the full data set (e.g. at the beginning of the notebook) and think about the adequate type of encoding (label or one-hot) for each of these columns. You find a description of the variables [here](https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot).
2. (Homework, optional): Retrain the model using your own, more nuanced encoding approach and try whether you can obtain a better predictive performance.

In [ ]:
# [cname for cname in X_train_full.columns if X_train_full[cname].dtype == "object"]

### Solution

__Suburb__ and __Adress__: no natural ordering, should be **one-hot encoded**

__Type:__ br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential. __The variable has no natural ordering --> use one-hot encoder__

__Method:__ S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available. __The variable has no natural ordering --> use one-hot encoder__

__SellerG:__ Real Estate Agent __The variable has no natural ordering --> use one-hot encoder__

__Date:__ Date sold __This is time-series data. It should be converted into a numerical variable (e.g by using [`pandas.to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)__).

__CouncilArea:__ Governing council for the area __The variable has no natural ordering --> use one-hot encoder__

__Regionname:__ General Region (West, North West, North, North east …etc)__The variable has no natural ordering --> use one-hot encoder__

# Exercise 4
Extend the pipeline by including the normalization step for the numerical features.

Copy this code to the original lecture notebook in order to execute it - it will not work here!

In [ ]:
""" # Preprocessing numerical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputation
    ('scaler', MinMaxScaler())                 # Normalization
])

# Preprocessing categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputation
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encoding
])

# Bundle both preprocessors
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),  # Apply numerical pipeline
    ('cat', categorical_transformer, low_cardinality_cols)  # Apply categorical pipeline
])

model = RandomForestRegressor(n_estimators=100, random_state=1)


# Bundle preprocessing and modeling code in a pipeline
complete_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Preprocess the raw training data and fit the model
complete_pipeline.fit(X_train, y_train)

# Preprocess the raw validation data and make predictions
preds = complete_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print("MAE using the complete pipeline: {}".format(score)) """